## 11/16
- TOPSとCLOTHINGの結合->T_Shirt_allへ
- 関数の作成(get_triplet)
### やること
- T_Shirt_all/trainとT_Shirt_all/testに分割
- 分割の仕方はランダムで行う？
    - ```train_ids=ids[np.random.choice(len(os.listdir),0.75)]``` 的な感じで75%を取り出せるかな？